### Reference:
https://stackoverflow.com/questions/77697302/how-to-run-ollama-in-google-colab

In [ ]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
# !sudo apt-get update && sudo apt-get install -y cuda-drivers

!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token("<your ngrok token>")

import os
import asyncio

# # Set LD_LIBRARY_PATH so the system NVIDIA library (for GPU)
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
from IPython.display import clear_output
clear_output()
run_process(['ollama', 'pull', 'mistral'])

In [ ]:
await asyncio.gather(
run_process(['ollama', 'serve']),
run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header="localhost:11434"']))

In [ ]:
# Use this in local IDE
llm = ChatOllama(model="mistral", temperature=0, base_url="<url from the output in above cell>")